In [1]:
import numpy as np
import csv
from sklearn.ensemble import RandomForestRegressor

# Find set of supercategories, categories 
supercategory_set = set()
category_set = set()
for i in range (10000):
	file = open("tags_train/" + str(i) + ".txt", "r")
	lines = file.readlines() 
	for  line in lines:
		words = line.strip().split(':')
		supercategory_set.add(words[0])
		category_set.add(words[1])
	file.close()

# mapping from (super) category to index
supercategory_dict = {item:val for val, item in enumerate(supercategory_set)}
category_dict = {item:val+1 for val, item in enumerate(category_set)}

# Vectorize train tags
train_tags = []
for i in range (10000):
	file = open("tags_train/" + str(i) + ".txt", "r")
	lines = file.readlines() 
	row = np.zeros(len(supercategory_set))
	for line in lines:
		words = line.strip().split(':')		
		supercategory_column = supercategory_dict.get(words[0])
		category_index = category_dict.get(words[1])
		row[supercategory_column] = category_index
	train_tags.append(row)
	file.close()

/anaconda3/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# Load train queries
query = []
with open('query_glove.csv', 'r') as csvfile:
	csv_reader = csv.reader(csvfile)
	for line in csv_reader:
		query.append(line)
print ("Finished loading queries")

Finished loading queries


In [3]:
# Reading 1000-d train features 
features_train = np.zeros((10000,1000))
with open('features_train/features_resnet1000_train.csv', 'r') as csvfile:
	csv_reader = csv.reader(csvfile)
	for line in csv_reader:
		image_name = line[0].strip(".jpg")[13:]
		row = []
		for i in range(len(line)):
			if i > 0:
				row.append(float(line[i]))		
		features_train[int(image_name)] = row

# Reading 2048-d train features 
features_train_2048 = np.zeros((10000,2048))
with open('features_train/features_resnet1000intermediate_train.csv', 'r') as csvfile:
	csv_reader = csv.reader(csvfile)
	for line in csv_reader:
		image_name = line[0].strip(".jpg")[13:]
		row = []
		for i in range(len(line)):
			if i > 0:
				row.append(float(line[i]))		
		features_train_2048[int(image_name)] = row

In [5]:
# Load test queries
query_test = []
with open('query_glove_test.csv', 'r') as csvfile:
	csv_reader = csv.reader(csvfile)
	for line in csv_reader:
		query_test.append(line)
print ("Finished loading queries")

# Vectorize test tags
test_tags = []
for i in range (2000):
	file = open("tags_test/" + str(i) + ".txt", "r")
	lines = file.readlines() 
	row = np.zeros(len(supercategory_set))
	for line in lines:
		words = line.strip().split(':')		
		supercategory_column = supercategory_dict.get(words[0])
		category_index = category_dict.get(words[1])
		row[supercategory_column] = category_index
	test_tags.append(row)
	file.close()

Finished loading queries


In [6]:
from sklearn.neighbors import KNeighborsRegressor
clf_knn = KNeighborsRegressor(n_neighbors=5)
features_train_concat = np.concatenate((np.array(features_train_2048),np.array(features_train)), axis=1)
features_train_concat = np.concatenate((features_train_concat, np.array(train_tags)), axis=1)
clf_knn.fit(query,features_train_concat)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [25]:
from sklearn.preprocessing import StandardScaler
clf_knn_std = KNeighborsRegressor(n_neighbors=5)
clf_knn_std.fit(StandardScaler().fit_transform(query), StandardScaler().fit_transform(features_train_concat))

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [26]:
features_test_2048 = np.zeros((2000,2048))
with open('features_test/features_resnet1000intermediate_test.csv', 'r') as csvfile:
	csv_reader = csv.reader(csvfile)
	for line in csv_reader:
		image_name = line[0].strip(".jpg")[12:]
		row = []
		for i in range(len(line)):
			if i > 0:
				row.append(float(line[i]))		
		features_test_2048[int(image_name)] = row

features_test_1000 = np.zeros((2000,1000))    
with open('features_test/features_resnet1000_test.csv', 'r') as csvfile:
	csv_reader = csv.reader(csvfile)
	for line in csv_reader:
		image_name = line[0].strip(".jpg")[12:]
		row = []
		for i in range(len(line)):
			if i > 0:
				row.append(float(line[i]))		
		features_test_1000[int(image_name)] = row
        
features_test_concat = np.concatenate((np.array(features_test_2048),np.array(features_test_1000)), axis=1)
features_test_concat = np.concatenate((features_test_concat, np.array(test_tags)), axis=1)

In [27]:
test_query_std = StandardScaler().fit_transform(np.array(query_test,dtype=float))
knn_predict_std = clf_knn_std.predict(test_query_std)
features_test_concat_std = StandardScaler().fit_transform(features_test_concat)
rank_list = []
for i in range (2000):
    guess = knn_predict_std[i]
    dist = []
    for j in range(2000):
        dist.append((np.linalg.norm(guess-features_test_concat_std[j]),j))
    dist.sort()
    rank_list.append([v for (k,v) in dist[:20]])

def mergeTwenty(lst):
    result = str(lst[0])+".jpg"
    for i in range(1,len(lst)):
        result += " " + str(lst[i]) + ".jpg"
    return result

with open('submission_knn_std.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=',')
    csv_writer.writerow(["Descritpion_ID", "Top_20_Image_IDs"])
    for i in range(len(rank_list)):
        csv_writer.writerow([str(i) + ".txt", mergeTwenty(rank_list[i])])

In [28]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics.scorer import make_scorer

def my_scorer(ground_truth, predictions):
    score = 0.0
    length = len(ground_truth)
    for i in range (length):
        dist = []
        for j in range(length):
            dist.append((np.linalg.norm(predictions[i]-ground_truth[j]),j))
        dist.sort()
        rank_list = [v for (k,v) in dist[:20]]
        if i in rank_list:
            rank = rank_list.index(i)
            score = score + (21.0-rank)/20.0
    return score/length

my_scorer = make_scorer(my_scorer, greater_is_better=True)
scores = cross_val_score(clf_knn_std, StandardScaler().fit_transform(query), StandardScaler().fit_transform(features_train_concat), cv=5,scoring=my_scorer)

In [23]:
scores_knn

array([0.38785 , 0.389425, 0.396675, 0.40325 , 0.3916  ])

In [29]:
scores_knn_std

array([0.344225, 0.35405 , 0.34785 , 0.3538  , 0.346425])